<a href="https://colab.research.google.com/github/matthiasdellago/Loss-Cartography/blob/main/Replicating%20%22Visualizing%20the%20Loss%20Landscape%20of%20Neural%20Nets'.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Download resnet.py and overwrite if it exists
!wget -O resnet.py https://raw.githubusercontent.com/tomgoldstein/loss-landscape/master/cifar10/models/resnet.py

# Import the resnet module
import resnet

center_model = resnet.ResNet18()

--2024-06-05 08:22:20--  https://raw.githubusercontent.com/tomgoldstein/loss-landscape/master/cifar10/models/resnet.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9883 (9.7K) [text/plain]
Saving to: ‘resnet.py’

resnet.py           100%[===================>]   9.65K  --.-KB/s    in 0.001s  

2024-06-05 08:22:20 (12.1 MB/s) - ‘resnet.py’ saved [9883/9883]



In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import CIFAR10
from torchvision import transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Use datatransform from paper
normalize = transforms.Normalize(mean=[x/255.0 for x in [125.3, 123.0, 113.9]],
                                     std=[x/255.0 for x in [63.0, 62.1, 66.7]])
# no data agumentation
transform = transforms.Compose([
    transforms.ToTensor(),
    normalize,
])

dataset = CIFAR10(root='./data', train=False, download=True, transform=transform) # 163 MB

BATCH_SIZE = 10 # Big but not too big, because I think that the batch is duplicated for each model in the vmap-enselmble
dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, num_workers = 2)

# criterion functional
criterion = F.cross_entropy

# parameter subspaces to investigate.
# for each string, the union of all modules that contain the string in their name will be considered.
# we will sample a random direction in that subspace + the gradient ascent and descent directions projected onto that subspace.
SUBSPACES = []

#check if we are on GPU, otherwise just proof of concept
if torch.cuda.is_available():
    NUM_DIRS = 0
    MAX_OOM = 0
    MIN_OOM = -7
else:
    NUM_DIRS = 0 # number of random directions to sample, in addition to the gradient ascent + descent, and radially in + out.
    MAX_OOM = 0 # maximum order of magnitude to sample
    MIN_OOM = -1 # minimum order of magnitude to sample

Files already downloaded and verified


Define utility functions for parameter space arithmetic. iadd is inplace, all others create a new model.

In [3]:
from copy import deepcopy

@torch.no_grad
def iadd(a:nn.Module, b:nn.Module):
    """add the parameters of b to a, inplace"""
    for a_param, b_param in zip(a.parameters(), b.parameters()):
        a_param.data.add_(b_param.data)
    return a

@torch.no_grad
def add(a_old:nn.Module, b:nn.Module):
    """add the parameters of b to a"""
    a = deepcopy(a_old)
    return iadd(a, b)

@torch.no_grad
def scale(a_old:nn.Module, b:float):
    """scale the parameters of a by b"""
    a = deepcopy(a_old)
    for a_param in a.parameters():
        a_param.data.mul_(b)
    return a

@torch.no_grad
def sub(a:nn.Module, b:nn.Module):
    """subtract the parameters of b from a"""
    neg_b = scale(b, -1)
    return iadd(neg_b, a)

@torch.no_grad
def abs(a:nn.Module):
    """return the norm of the parameters of a"""
    return torch.norm(torch.cat([param.data.flatten() for param in a.parameters()]))

@torch.no_grad
def norm(a:nn.Module):
    """normalize the parameters of a"""
    return scale(a, 1/abs(a))

@torch.no_grad
def rand_like(a: nn.Module):
    """normalised random direction in the parameter space of the model."""
    rand = deepcopy(a)
    for param in rand.parameters():
        param.data = torch.randn_like(param.data)
    return norm(rand)

@torch.no_grad
def project_to_module(a: nn.Module, target_subspace: str):
    """
    normalised projection onto the subspace of the model specified by `target_subspace`.
    subspace: Any parameter whose name contains the `target_subspace` string.
    """
    projection = deepcopy(a)
    for name, param in projection.named_parameters():
        if target_subspace not in name:
            param.data.zero_()
    return norm(projection)

Define profiling context manager: measure execution time and GPU RAM usage before and after.

In [4]:
from time import perf_counter
from contextlib import contextmanager
import torch

@contextmanager
def profiler(description, length=80, pad_char=':'):
    print('\n'+description.center(length, pad_char))
    if torch.cuda.is_available():
        print(f'GPU RAM before execution: Allocated: {torch.cuda.memory_allocated()/1e9:.2f} GB | Reserved: {torch.cuda.memory_reserved()/1e9:.2f} GB')
    start = perf_counter()
    yield
    if torch.cuda.is_available():
        print(f'GPU RAM after execution:  Allocated: {torch.cuda.memory_allocated()/1e9:.2f} GB | Reserved: {torch.cuda.memory_reserved()/1e9:.2f} GB')
    print(f'Finished {description} in {perf_counter() - start:.2f} s'.center(length, pad_char))

with profiler('Profiler example'):
    print('Computation goes here...')


::::::::::::::::::::::::::::::::Profiler example::::::::::::::::::::::::::::::::
GPU RAM before execution: Allocated: 0.00 GB | Reserved: 0.00 GB
Computation goes here...
GPU RAM after execution:  Allocated: 0.00 GB | Reserved: 0.00 GB
::::::::::::::::::::::Finished Profiler example in 0.00 s:::::::::::::::::::::::


In [5]:
# create a model to find the gradient
grad_model = deepcopy(center_model)
grad_model = grad_model.to(device)

optimizer = torch.optim.SGD(grad_model.parameters(), lr=1)
optimizer.zero_grad()
torch.set_grad_enabled(True)

# take a single step of gradient descent
# gradient accumulate over the whole dataset
with profiler('1 step of gradient descent over the whole dataset'):
    for data, target in dataloader:
        data, target = data.to(device), target.to(device)
        output = grad_model(data)
        loss = criterion(output, target)
        loss.backward()

optimizer.step()
# from this point on we wont need any gradients
torch.set_grad_enabled(False)


# calculate the direction of gradient decent from the center model
grad_model = grad_model.to('cpu')
dir_ascent = sub(center_model, grad_model)
dir_ascent = norm(dir_ascent)
dir_descent = scale(dir_ascent, -1)

del grad_model



:::::::::::::::1 step of gradient descent over the whole dataset::::::::::::::::
GPU RAM before execution: Allocated: 0.04 GB | Reserved: 0.07 GB
GPU RAM after execution:  Allocated: 0.11 GB | Reserved: 0.21 GB
:::::Finished 1 step of gradient descent over the whole dataset in 16.67 s::::::


In [6]:
radial = norm(center_model)

dirs = {
    'Ascent': dir_ascent,
    'Descent': dir_descent,
    'Radially Out': radial,
    'Radially In': scale(radial,-1),
    }

# add random directions
dirs.update({f'Random {i}': rand_like(center_model) for i in range(NUM_DIRS)})

# add projections of each dir in dirs onto each SUBSPACE
# Add projections of each direction in dirs onto each subspace
if not SUBSPACES == []:
    for dir_name, direction in list(dirs.items()):
        for subspace in SUBSPACES:
            projected_dir = project_to_module(direction, subspace)
            new_key = f'{dir_name} ⋅ P({subspace})'
            dirs[new_key] = projected_dir

# check that they are all normalised
assert all(torch.isclose(abs(d), torch.tensor(1.0), rtol=0.001) for d in dirs.values())

In [7]:
import pandas as pd
import numpy as np

# convert to base 2
POW_MIN_DIST = int(np.floor(MIN_OOM*np.log2(10)))
POW_MAX_DIST = int(np.ceil(MAX_OOM*np.log2(10)))

# will sample these distances
scales = 2. ** np.arange(POW_MIN_DIST, POW_MAX_DIST + 1)

# offset each direction by a different amount, so that we get a nice spread of distances, to avoid artefacts/robustness
offset_per_direction = 2. ** np.linspace(0, 1, len(dirs), endpoint=False)

# Initialize an empty list to hold tuples for MultiIndex
index_list = []

# Calculate distances using broadcasting
distances = np.outer(scales, offset_per_direction).T.flatten()

# Repeat directions accordingly
directions = np.repeat(list(dirs.keys()), len(scales))

# Create DataFrame directly from numpy arrays
df = pd.DataFrame(index=pd.MultiIndex.from_arrays([directions, distances], names=['Direction', 'Distance']))

df


Empty DataFrame
Columns: []
Index: [(Ascent, 5.960464477539063e-08), (Ascent, 1.1920928955078125e-07), (Ascent, 2.384185791015625e-07), (Ascent, 4.76837158203125e-07), (Ascent, 9.5367431640625e-07), (Ascent, 1.9073486328125e-06), (Ascent, 3.814697265625e-06), (Ascent, 7.62939453125e-06), (Ascent, 1.52587890625e-05), (Ascent, 3.0517578125e-05), (Ascent, 6.103515625e-05), (Ascent, 0.0001220703125), (Ascent, 0.000244140625), (Ascent, 0.00048828125), (Ascent, 0.0009765625), (Ascent, 0.001953125), (Ascent, 0.00390625), (Ascent, 0.0078125), (Ascent, 0.015625), (Ascent, 0.03125), (Ascent, 0.0625), (Ascent, 0.125), (Ascent, 0.25), (Ascent, 0.5), (Ascent, 1.0), (Descent, 7.08822676541043e-08), (Descent, 1.417645353082086e-07), (Descent, 2.835290706164172e-07), (Descent, 5.670581412328344e-07), (Descent, 1.1341162824656687e-06), (Descent, 2.2682325649313374e-06), (Descent, 4.536465129862675e-06), (Descent, 9.07293025972535e-06), (Descent, 1.81458605194507e-05), (Descent, 3.62917210389014e-05), (Descent, 7.25834420778028e-05), (Descent, 0.0001451668841556056), (Descent, 0.0002903337683112112), (Descent, 0.0005806675366224224), (Descent, 0.0011613350732448448), (Descent, 0.0023226701464896895), (Descent, 0.004645340292979379), (Descent, 0.009290680585958758), (Descent, 0.018581361171917516), (Descent, 0.03716272234383503), (Descent, 0.07432544468767006), (Descent, 0.14865088937534013), (Descent, 0.29730177875068026), (Descent, 0.5946035575013605), (Descent, 1.189207115002721), (Radially Out, 8.429369702178807e-08), (Radially Out, 1.6858739404357614e-07), (Radially Out, 3.371747880871523e-07), (Radially Out, 6.743495761743046e-07), (Radially Out, 1.3486991523486091e-06), (Radially Out, 2.6973983046972182e-06), (Radially Out, 5.3947966093944364e-06), (Radially Out, 1.0789593218788873e-05), (Radially Out, 2.1579186437577746e-05), (Radially Out, 4.315837287515549e-05), (Radially Out, 8.631674575031098e-05), (Radially Out, 0.00017263349150062197), (Radially Out, 0.00034526698300124393), (Radially Out, 0.0006905339660024879), (Radially Out, 0.0013810679320049757), (Radially Out, 0.0027621358640099515), (Radially Out, 0.005524271728019903), (Radially Out, 0.011048543456039806), (Radially Out, 0.02209708691207961), (Radially Out, 0.04419417382415922), (Radially Out, 0.08838834764831845), (Radially Out, 0.1767766952966369), (Radially Out, 0.3535533905932738), (Radially Out, 0.7071067811865476), (Radially Out, 1.4142135623730951), (Radially In, 1.0024266424819404e-07), (Radially In, 2.0048532849638808e-07), (Radially In, 4.0097065699277616e-07), (Radially In, 8.019413139855523e-07), (Radially In, 1.6038826279711046e-06), (Radially In, 3.2077652559422093e-06), (Radially In, 6.4155305118844185e-06), (Radially In, 1.2831061023768837e-05), (Radially In, 2.5662122047537674e-05), (Radially In, 5.132424409507535e-05), (Radially In, 0.0001026484881901507), (Radially In, 0.0002052969763803014), (Radially In, 0.0004105939527606028), (Radially In, 0.0008211879055212056), (Radially In, 0.0016423758110424111), (Radially In, 0.0032847516220848223), (Radially In, 0.0065695032441696445), (Radially In, 0.013139006488339289), (Radially In, 0.026278012976678578), (Radially In, 0.052556025953357156), (Radially In, 0.10511205190671431), (Radially In, 0.21022410381342863), (Radially In, 0.42044820762685725), (Radially In, 0.8408964152537145), (Radially In, 1.681792830507429)]

[100 rows x 0 columns]

In [8]:
# fill the dataframe with the models
for dir_name, dist in df.index:
    shift = scale(dirs[dir_name], dist)     # shift = direction * distance
    location = iadd(shift, center_model)    # location = center + shift
    df.loc[(dir_name, dist), 'Model'] = location

df

Model
Direction   Distance                                                       
Ascent      5.960464e-08  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...
            1.192093e-07  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...
            2.384186e-07  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...
            4.768372e-07  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...
            9.536743e-07  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...
...                                                                     ...
Radially In 1.051121e-01  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...
            2.102241e-01  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...
            4.204482e-01  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...
            8.408964e-01  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...
            1.681793e+00  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...

[100 rows x 1 columns]

Parallel evaluation with vmap()


In [9]:
from torch.func import stack_module_state, functional_call
from torch import vmap

# add the center model
ensemble_list = [center_model] + list(df['Model'])

print(f'Ensemble size: {len(ensemble_list)}')

# stack to prepare for vmap
params, buffers = stack_module_state(ensemble_list)

with profiler(f'Moving stacked ensemble to {device}'):
    params = {name: tensor.to(device) for name, tensor in params.items()}
    buffers = {name: tensor.to(device) for name, tensor in buffers.items()}

stacked_ensemble = (params, buffers)

# Construct a "stateless" version of one of the models. It is "stateless" in
# the sense that the parameters are meta Tensors and do not have storage.
meta_model = deepcopy(center_model).to('meta')

def meta_model_loss(params_and_buffers, data, target):
    """Compute the loss of a set of params on a batch of data, via the meta model"""
    predictions = functional_call(meta_model, params_and_buffers, (data,))
    predictions = predictions.double() # double precision for better resoltion of small scales
    loss = criterion(predictions, target)
    return loss

# define a loss function that takes the stacked ensemble params, data, and target
# in_dims=(0, None, None) adds an ensemble dimension to the first argument 'params_and_buffers' but not to the other two arguments
vmap_loss = vmap(meta_model_loss, in_dims=(0, None, None))

batch_losses = []
with profiler(f'Evaluating stacked ensemble on {device}'):
    for data, target in dataloader:
        data, target = data.to(device), target.to(device)
        batch_loss = vmap_loss(stacked_ensemble, data, target)
        batch_losses.append(batch_loss)

with profiler('Freeing data and target'):
    del data, target
    torch.cuda.empty_cache()

with profiler('Freeing stacked ensemble'):
    del stacked_ensemble
    torch.cuda.empty_cache()

loss_tensor = torch.mean(torch.stack(batch_losses), dim=0)

Ensemble size: 101

::::::::::::::::::::::::Moving stacked ensemble to cuda:::::::::::::::::::::::::
GPU RAM before execution: Allocated: 0.02 GB | Reserved: 0.21 GB
GPU RAM after execution:  Allocated: 4.54 GB | Reserved: 4.62 GB
:::::::::::::::Finished Moving stacked ensemble to cuda in 1.01 s:::::::::::::::

::::::::::::::::::::::Evaluating stacked ensemble on cuda:::::::::::::::::::::::
GPU RAM before execution: Allocated: 4.54 GB | Reserved: 4.62 GB
GPU RAM after execution:  Allocated: 4.54 GB | Reserved: 6.81 GB
::::::::::::Finished Evaluating stacked ensemble on cuda in 592.37 s::::::::::::

::::::::::::::::::::::::::::Freeing data and target:::::::::::::::::::::::::::::
GPU RAM before execution: Allocated: 4.54 GB | Reserved: 6.81 GB
GPU RAM after execution:  Allocated: 4.54 GB | Reserved: 4.61 GB
:::::::::::::::::::Finished Freeing data and target in 0.07 s:::::::::::::::::::

::::::::::::::::::::::::::::Freeing stacked ensemble::::::::::::::::::::::::::::
GPU RAM before execu

In [10]:
# Convert the loss tensor to a list and unpack
center_loss, *dir_losses = loss_tensor.tolist()

# Ensure the length of dir_losses matches the DataFrame length
assert len(dir_losses) == len(df)

# Add the directional losses to the DataFrame
df['Loss'] = dir_losses

# Add the center_loss to each Direction, for Distance=0
for direction in df.index.get_level_values('Direction').unique():
    df.loc[(direction, 0.0), 'Loss'] = center_loss

df.sort_index(inplace=True)

df

Model  \
Direction    Distance                                                          
Ascent       0.000000e+00                                                NaN   
             5.960464e-08  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             1.192093e-07  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             2.384186e-07  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             4.768372e-07  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
...                                                                      ...   
Radially Out 8.838835e-02  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             1.767767e-01  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             3.535534e-01  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             7.071068e-01  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             1.414214e+00  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   

                               Loss  
Direction    Distance                
Ascent       0.000000e+00  2.387065  
             5.960464e-08  2.387065  
             1.192093e-07  2.387065  
             2.384186e-07  2.387066  
             4.768372e-07  2.387066  
...                             ...  
Radially Out 8.838835e-02  2.387461  
             1.767767e-01  2.387859  
             3.535534e-01  2.388658  
             7.071068e-01  2.390270  
             1.414214e+00  2.393555  

[104 rows x 2 columns]

In [11]:
import plotly.graph_objects as go

# Create a Plotly figure
fig = go.Figure()

# Plotting directly from grouped data
for direction, data in df.groupby(level='Direction'):
    fig.add_trace(go.Scatter(
        x=data.index.get_level_values('Distance'),
        y=data['Loss'],
        mode='lines+markers',
        name=direction
    ))

# Set axis labels and title with adjusted figure dimensions
fig.update_layout(
    title=f'Loss Landscape of {center_model.__class__.__name__} on {dataset.__class__.__name__}',
    xaxis_title='Distance from Center in Parameter Space',
    yaxis_title='Loss',
    legend_title='Direction',
    template='seaborn',
    height=600,
)


# Show the figure
fig.show()


Now let's measure the roughness.

In [12]:
# TODO: Add nice explanation of the roughness calculation
df['Roughness'] = np.nan

# Calculate the roughness, one direction at a time
for direction, group in df.groupby(level='Direction'):
    distA = group.index.get_level_values('Distance').to_numpy()
    losses = group['Loss'].to_numpy()

    # Named slices, so that
    # distA[x] really corresponds to the distance between A and X ∈ {A, B, C},
    # losses[x] really corresponds to the loss at X ∈ {A, B, C}
    a = 0
    b = slice(1, -1)
    c = slice(2, None)

    # Ensure distances and slices are correct
    assert np.all(distA[c] == 2 * distA[b]), "Distance check failed"

    # Calculate paths between A, B, and C
    AB = np.sqrt((losses[b] - losses[a]) ** 2 + distA[b] ** 2)
    BC = np.sqrt((losses[c] - losses[b]) ** 2 + distA[b] ** 2)
    AC = np.sqrt((losses[c] - losses[a]) ** 2 + distA[c] ** 2)

    # Roghness is how much the path via B (A->B->C) is longer than the direct path A->C
    # subtracting 1, then we can use log scale better!
    roughness = (AB + BC) / AC - 1

    # Update DataFrame with roughness values
    df.loc[(direction,), 'Roughness'] = np.concatenate([[np.nan], roughness, [np.nan]])

df

Model  \
Direction    Distance                                                          
Ascent       0.000000e+00                                                NaN   
             5.960464e-08  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             1.192093e-07  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             2.384186e-07  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             4.768372e-07  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
...                                                                      ...   
Radially Out 8.838835e-02  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             1.767767e-01  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             3.535534e-01  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             7.071068e-01  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             1.414214e+00  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   

                               Loss     Roughness  
Direction    Distance                              
Ascent       0.000000e+00  2.387065           NaN  
             5.960464e-08  2.387065  2.759516e-03  
             1.192093e-07  2.387065  7.050656e-04  
             2.384186e-07  2.387066  2.869951e-04  
             4.768372e-07  2.387066  6.858418e-04  
...                             ...           ...  
Radially Out 8.838835e-02  2.387461  2.453993e-11  
             1.767767e-01  2.387859  9.411427e-11  
             3.535534e-01  2.388658  3.872451e-10  
             7.071068e-01  2.390270  1.575317e-09  
             1.414214e+00  2.393555           NaN  

[104 rows x 3 columns]

In [13]:
# new nomenclature: grit := scale dependent roughness

# Create a Plotly figure
fig = go.Figure()

# Plotting roughness for each direction using grouped data
for direction, data in df.groupby(level='Direction'):
    fig.add_trace(go.Scatter(
        x=data.index.get_level_values('Distance'),
        y=data['Roughness'],
        mode='markers',
        name=direction
    ))

# Set axis labels, title, and configure log scale on x-axis
fig.update_layout(
    title=f'Scale Dependent Roughness of {center_model.__class__.__name__} on {dataset.__class__.__name__}',
    xaxis={
        'title': 'Coarse Graining Scale',
        'type': 'log',
        'dtick': 1,  # Tick every power of ten
    },
    yaxis ={
        'title': 'Roughness',
        'type': 'log',
    },
    legend_title='Direction',
    template='seaborn',
    height=600
)

# Display the figure
fig.show()
